In [1]:
import pyodbc
import pandas as pd
from datetime import datetime

In [2]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

StoreMaster - List of all the active stores in the DaburGT Universe

In [3]:
StoreMasterQuery = """
SELECT 
-- SM.ClientStoreCode,
SM.LISStoreCode,SM.StoreName,
RIGHT(SM.ChainCode,(LEN(SM.ChainCode)-LEN('DBRPHGT'))) AS ChainCode,SM.StoreState,SM.StoreCity, sm.StateRegion 
-- ,SM.IsEnabled, SM.StoreStatus
FROM [PowerBI_LISV2].[DaburPharmaManagement].StoreMaster SM
WHERE  
SM.ChainCode IN ('DBRPHGTHIND','DBRPHGTPRAGATI') AND (SM.StoreStatus IS NULL OR SM.StoreStatus = 'Active')
AND (SM.StoreName <> 'TEST' OR SM.StoreName NOT LIKE '%Test%')
AND SM.IsEnabled = 'True'
"""

#### VisibilityQuery 
POSM Availbility vs Audit Responses from AuditiInput dataset

In [4]:
VisibilityQuery = """
SELECT DISTINCT VA.visitid,VA.[submitteddate&time],
                VA.supervisorcode,
                VA.username,VA.fullname,VA.designation,
                VA.lisstorecode,
                VA.pjpstatus,
                va.posmname,
                VA.posmbrand,
                VA.posmstartdate,
                VA.posmenddate,
                -- VA.AuditStatus,
                CASE
                  WHEN IK.auditstatus = 'Audit' THEN 'Yes'
                  ELSE 'No'
                END AS Audit_Status,
                VA.visibilityavailable,
                VA.nonavailabilityreason,
                VA.prephotolink,VA.postcloseupphotolink,VA.postdistancephotolink
FROM   [PowerBI_LISV2].[DaburPharmaManagement].visibilityappinputreport AS VA
       LEFT JOIN (SELECT visitid,auditstatus
                  FROM [PowerBI_LISV2].[DaburPharmaManagement].visibilityauditinputreport
                  GROUP  BY visitid,auditstatus) AS IK
              ON IK.visitid = VA.visitid
WHERE  VA.month = Month(Getdate() - 1)
       AND VA.year = Year(Getdate() - 1)
       AND VA.username NOT LIKE '%Star%'
       AND VA.username NOT LIKE '%SUP%'
       AND VA.auditstatus IN ( 'Pending', 'Yes' ) 
"""

In [5]:

AppResponseQuery = """
SELECT *
FROM (
    SELECT DISTINCT
        VA.VisitId,
        VA.AppQuestions,
        VA.AppResponse
    FROM [PowerBI_LISV2].[DaburPharmaManagement].VisibilityAppInputReport AS VA
    WHERE [MONTH] = MONTH(GETDATE()-1) AND [YEAR] = YEAR(GETDATE()-1) 
) AS SourceTable
PIVOT (
    MAX(AppResponse)
    FOR AppQuestions IN (
        [Is Stock available?],
        [Stock as per Planogram],
        [POSM Availability]
    )
) AS PivotResult;
"""

In [6]:
conn = pyodbc.connect(conn_str)

StoreMaster = pd.read_sql(StoreMasterQuery, conn)
Visibility  = pd.read_sql(VisibilityQuery, conn)
AppResposne = pd.read_sql(AppResponseQuery, conn)
conn.close()

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_19184\2633812236.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  StoreMaster = pd.read_sql(StoreMasterQuery, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_19184\2633812236.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Visibility  = pd.read_sql(VisibilityQuery, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_19184\2633812236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AppResposne = pd.read_sql(AppResponseQuery, conn)


In [7]:
AppResposne.head()

VisitId Is Stock available? Stock as per Planogram POSM Availability
0   490320                 Yes                    YES      Shelf Strips
1   490322                  No                     NO      Shelf Strips
2   490345                 Yes                    YES        Window Kit
3   490347                 Yes                    YES      Shelf Strips
4   490354                 Yes                    YES      Shelf Strips

In [8]:
VisibilityAppResponse = pd.merge(Visibility,AppResposne, on = 'VisitId' , how = 'inner')

In [9]:
VisibilityAppResponse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11136 entries, 0 to 11135
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   VisitId                 11136 non-null  int64         
 1   SubmittedDate&Time      11136 non-null  datetime64[ns]
 2   SupervisorCode          11135 non-null  object        
 3   Username                11136 non-null  object        
 4   FullName                11136 non-null  object        
 5   Designation             11136 non-null  object        
 6   LISStoreCode            11136 non-null  object        
 7   PJPStatus               11136 non-null  object        
 8   POSMName                11136 non-null  object        
 9   POSMBrand               11136 non-null  object        
 10  POSMStartDate           11136 non-null  object        
 11  POSMEndDate             11136 non-null  object        
 12  Audit_Status            11136 non-null  object

In [27]:
StoreMaster['ChainCode'].value_counts()

ChainCode
PRAGATI    5571
HIND       3812
Name: count, dtype: int64

In [28]:
# VisibilityAppResponse.head()

In [20]:
VisibilityAppResponse.to_csv(r'C:\Users\Administrator\LIS\Dabur\VisibilityAppResponse.csv', index=False)
StoreMaster.to_csv(r'C:\Users\Administrator\LIS\Dabur\StoreMaster.csv', index=False)

In [21]:
VisibilityAppResponse.head()

VisitId  SubmittedDate&Time SupervisorCode    Username  \
0   490331 2025-06-02 09:40:27    DBRPH-SUP-6  DBR-ISP297   
1   490334 2025-06-02 09:41:44    DBRPH-SUP-3  DBR-ISP150   
2   490341 2025-06-02 09:57:59    DBRPH-SUP-7  DBR-ISP147   
3   490350 2025-06-02 10:05:09    DBRPH-SUP-7  DBR-ISP232   
4   490354 2025-06-02 10:05:17    DBRPH-SUP-3  DBR-ISP202   

                     FullName   Designation                    LISStoreCode  \
0                       Vicky  Merchandiser            DBRPH-HYD0TG01494870   
1  Thorat Jiteshbhai Babubhai  Merchandiser            DBRPH-SRT0GJ00192261   
2        Sonai Muthaiya Ramar  Merchandiser  DBRPH-0022008630MDR0TN00291094   
3                  Harivishnu  Merchandiser            DBRPH-COI0TN11526929   
4      Mahendra Suresh Jadhav  Merchandiser            DBRPH-MUM0MH11639050   

   PJPStatus        POSMName      POSMBrand  ... POSMEndDate Audit_Status  \
0    Planned  Window Display      ORAL CARE  ...  30-06-2025          Yes   
1    Planned  Window Display        GLUCOSE  ...  30-06-2025          Yes   
2  UnPlanned  Window Display      ORAL CARE  ...  30-06-2025          Yes   
3  UnPlanned  Window Display          Honey  ...  30-06-2025          Yes   
4    Planned  Window Display  Amla Hair Oil  ...  30-06-2025          Yes   

  VisibilityAvailable NonAvailabilityReason  \
0                 Yes                  None   
1                 Yes                  None   
2                 Yes                  None   
3                 Yes                  None   
4                 Yes                  None   

                                        PrePhotoLink  \
0  https://lisimagev2.loveinstore.com//LISImages/...   
1  https://lisimagev2.loveinstore.com//LISImages/...   
2  https://lisimagev2.loveinstore.com//LISImages/...   
3  https://lisimagev2.loveinstore.com//LISImages/...   
4  https://lisimagev2.loveinstore.com//LISImages/...   

                                PostCloseUpPhotoLink  \
0  https://lisimagev2.loveinstore.com//LISImages/...   
1  https://lisimagev2.loveinstore.com//LISImages/...   
2  https://lisimagev2.loveinstore.com//LISImages/...   
3  https://lisimagev2.loveinstore.com//LISImages/...   
4  https://lisimagev2.loveinstore.com//LISImages/...   

                               PostDistancePhotoLink Is Stock available?  \
0  https://lisimagev2.loveinstore.com//LISImages/...                 Yes   
1  https://lisimagev2.loveinstore.com//LISImages/...                 Yes   
2  https://lisimagev2.loveinstore.com//LISImages/...                 Yes   
3  https://lisimagev2.loveinstore.com//LISImages/...                 Yes   
4  https://lisimagev2.loveinstore.com//LISImages/...                 Yes   

  Stock as per Planogram POSM Availability  
0                    YES      Shelf Strips  
1                    YES        Window Kit  
2                    YES      Shelf Strips  
3                    YES      Shelf Strips  
4                    YES      Shelf Strips  

[5 rows x 21 columns]

#### Coverage Data vs No Permission 

In [15]:
AttendanceNormalExportQuery = """
WITH Attendance_NormalExport AS (
    SELECT DISTINCT
        FORMAT(CONVERT(DATETIME, VisitDate, 105), 'dd-MM-yyyy') +'-'+LISStoreCode +'-'+ Username  AS VisibilityKey,
        LISStoreCode,UserName,FORMAT(CONVERT(DATETIME, VisitDate, 105), 'dd-MM-yyyy') AS VisitDate,PJPStatus, 
        [CheckIn-Time], [CheckOut-Time],AttendanceType,NoPermissionReason,
        CASE WHEN AttendanceType = 'Attendance' THEN 'Y'
        	 WHEN AttendanceType = 'NoPermission' THEN 'Y' ELSE 'NA' END AS 'VisitCovered'
    FROM [PowerBI_LISV2].[DaburPharmaManagement].ISPStoreAttendance_NormalExport
    WHERE MONTH = 6 AND YEAR = 2025 AND (UserName NOT LIKE '%Star%' AND UserName NOT LIKE '%SUP%')
    -- AND LISStoreCode = 'DBRPH-ARA0TN11603966'
    -- AND UserName = 'DBR-ISP101' 
    -- and FORMAT(CONVERT(DATETIME, VisitDate, 105), 'dd-MM-yyyy') = '02-06-2025'
)
SELECT NE.* , ISNULL(VAPINPUT.Merchandised,'N') AS Merchandised
FROM Attendance_NormalExport AS NE
LEFT JOIN (SELECT DISTINCT 
    FORMAT(CONVERT(DATETIME, [SubmittedDate&Time], 105), 'dd-MM-yyyy') +'-'+LISStoreCode +'-'+ Username  AS VisibilityKey,
	'Y' as Merchandised
FROM [PowerBI_LISV2].[DaburPharmaManagement].VisibilityAppInputReport
where MONTH = 6 AND YEAR = 2025
) AS VAPINPUT ON VAPINPUT.VisibilityKey = ne.VisibilityKey
;
"""

In [16]:
conn = pyodbc.connect(conn_str)

AttendanceNormalExport = pd.read_sql(AttendanceNormalExportQuery, conn)
#AttendanceNormalExport is used to understand the coverage, merchandised and no merchandised reasons

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_19184\1366849453.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AttendanceNormalExport = pd.read_sql(AttendanceNormalExportQuery, conn)


In [17]:
AttendanceNormalExport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16764 entries, 0 to 16763
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   VisibilityKey       16764 non-null  object
 1   LISStoreCode        16764 non-null  object
 2   UserName            16764 non-null  object
 3   VisitDate           16764 non-null  object
 4   PJPStatus           16764 non-null  object
 5   CheckIn-Time        16764 non-null  object
 6   CheckOut-Time       16764 non-null  object
 7   AttendanceType      16764 non-null  object
 8   NoPermissionReason  7637 non-null   object
 9   VisitCovered        16764 non-null  object
 10  Merchandised        16764 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


In [18]:
AttendanceNormalExport.to_csv(r'C:\Users\Administrator\LIS\Dabur\CoverageMerchandising.csv', index=False)

In [19]:
StoreMaster.head()

LISStoreCode             StoreName ChainCode StoreState  \
0  DBRPH-PTA0BI11622917            SUBH KIRNA   PRAGATI      Bihar   
1  DBRPH-BBR0OR00909365      SAI DEEPAK STORE   PRAGATI     Odisha   
2  DBRPH-BBR0OR00914492           SABAT STORE   PRAGATI     Odisha   
3  DBRPH-BBR0OR11665698  BHOLANATH BOOK STORE   PRAGATI     Odisha   
4  DBRPH-BBR0OR11676349       DIPUN VER STORE   PRAGATI     Odisha   

     StoreCity StateRegion  
0        Patna       North  
1  Bhubaneswar        East  
2  Bhubaneswar        East  
3  Bhubaneswar        East  
4  Bhubaneswar        East